In [90]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import os

In [91]:
full_game_df = pd.read_csv(os.path.join("data","combined_game_data.csv"))

In [92]:
sig_cols = list(np.load("sig_cols.npy"))

In [93]:
full_game_df.head()

,Unnamed: 0,G,Date,Age,Tm,game_location,Opp,game_result,GS,MP,...,Career-Stage,Above-Avg-PTS-Per-POSS,More-BLKs-Than-STLs,More-AST-Than-TOV,PTS-Cat,2P,2PA,2P%,2P%-Above-50%,More than 3 Fouls
0,0,1,2003-10-29,18,CLE,0,SAC,-14,1.0,42,...,rising,True,False,True,below 35,12.0,18.0,0.666667,True,False
1,1,2,2003-10-30,18,CLE,0,PHO,-9,1.0,40,...,rising,False,False,True,below 35,7.0,12.0,0.583333,True,False
2,2,3,2003-11-01,18,CLE,0,POR,-19,1.0,39,...,rising,False,False,True,below 20,3.0,11.0,0.272727,False,False
3,3,4,2003-11-05,18,CLE,1,DEN,-4,1.0,41,...,rising,False,True,True,below 20,3.0,9.0,0.333333,False,False
4,4,5,2003-11-07,18,CLE,0,IND,-1,1.0,43,...,rising,False,False,False,below 35,7.0,16.0,0.437500,False,False


In [94]:
sig_df = full_game_df[sig_cols]

In [95]:
sig_df.head()

,game_result,FG%,GmSc,TS%,GmSc-Per-PTS,PTS-Per-POSS,game_result-Per-PTS,PER,game_location,Opp,...,Over-3-BLKs-and-STLs,Played-Above-Avg-Min,Above-Avg-FT%,Career-Stage,Above-Avg-PTS-Per-POSS,More-BLKs-Than-STLs,More-AST-Than-TOV,PTS-Cat,2P%-Above-50%,More than 3 Fouls
0,-14,0.600,24.7,0.583431,0.988000,1.172608,-0.560000,29.277048,0,SAC,...,True,True,False,rising,True,False,True,below 35,True,False
1,-9,0.471,14.7,0.516605,0.700000,0.837321,-0.428571,16.893600,0,PHO,...,False,True,False,rising,False,False,True,below 35,True,False
2,-19,0.250,5.0,0.308880,0.625000,0.537634,-2.375000,5.489179,0,POR,...,False,True,True,rising,False,False,True,below 20,False,False
3,-4,0.273,11.2,0.305011,1.600000,0.611888,-0.571429,13.291049,1,DEN,...,True,True,True,rising,False,True,True,below 20,False,False
4,-1,0.444,9.0,0.539273,0.391304,0.819088,-0.043478,8.698953,0,IND,...,False,True,True,rising,False,False,False,below 35,False,False


In [96]:
sig_df.replace(True,1,inplace=True)
sig_df.replace(False,0,inplace=True)

c:\users\kambo\.virtualenvs\lebronjames-q3tyihsg\lib\site-packages\pandas\core\frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [97]:
sig_df.select_dtypes(include=["int64"]).columns

Index(['game_result', 'game_location'], dtype='object')

In [98]:
for col in sig_df.select_dtypes(include=["int64"]).columns:
    sig_df[col] = sig_df[col].apply(lambda x: float(x))

c:\users\kambo\.virtualenvs\lebronjames-q3tyihsg\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [99]:
sig_df.dtypes

game_result               float64
FG%                       float64
GmSc                      float64
TS%                       float64
GmSc-Per-PTS              float64
PTS-Per-POSS              float64
game_result-Per-PTS       float64
PER                       float64
game_location             float64
Opp                        object
Won                       float64
High-3P%-Shooting         float64
High-FG%-Shooting         float64
Over-3-BLKs-and-STLs      float64
Played-Above-Avg-Min      float64
Above-Avg-FT%             float64
Career-Stage               object
Above-Avg-PTS-Per-POSS    float64
More-BLKs-Than-STLs       float64
More-AST-Than-TOV         float64
PTS-Cat                    object
2P%-Above-50%             float64
More than 3 Fouls         float64
dtype: object

In [100]:
def label_bin(cat_col,df):
    lb = LabelBinarizer()
    data = lb.fit_transform(df[cat_col])
    classes = lb.classes_
    new_df = pd.DataFrame(data,columns=classes,dtype=float)
    for col in new_df.columns:
        df[col] = new_df[col]
    return lb

In [101]:
label_bins = []
for object_col in sig_df.select_dtypes(include=["object"]).columns:
    object_lb = label_bin(object_col,sig_df)
    object_col_lb_pair = (object_col,object_lb)
    label_bins.append(object_col_lb_pair)

c:\users\kambo\.virtualenvs\lebronjames-q3tyihsg\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\users\kambo\.virtualenvs\lebronjames-q3tyihsg\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\users\kambo\.virtualenvs\lebronjames-q3tyihsg\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [102]:
sig_df.tail()

,game_result,FG%,GmSc,TS%,GmSc-Per-PTS,PTS-Per-POSS,game_result-Per-PTS,PER,game_location,Opp,...,SEA,TOR,UTA,WAS,decline,prime,rising,above 35,below 20,below 35
1255,10.0,0.571,31.3,0.657778,0.845946,1.170886,0.270270,41.786472,1.0,MIL,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1256,9.0,0.412,25.8,0.591966,0.921429,1.208981,0.321429,36.256500,0.0,LAC,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1257,-2.0,0.545,24.7,0.594872,0.851724,1.106870,-0.068966,35.312382,1.0,BRK,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1258,2.0,0.316,9.6,0.401003,0.600000,0.699301,0.125000,12.610471,1.0,LAC,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1259,-15.0,0.467,14.3,0.545703,0.715000,0.948767,-0.750000,19.701088,0.0,TOR,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [103]:
label_bins

[('Opp', LabelBinarizer()),
 ('Career-Stage', LabelBinarizer()),
 ('PTS-Cat', LabelBinarizer())]

In [104]:
sig_df.select_dtypes(include=["object"])

,Opp,Career-Stage,PTS-Cat
0,SAC,rising,below 35
1,PHO,rising,below 35
2,POR,rising,below 20
3,DEN,rising,below 20
4,IND,rising,below 35
...,...,...,...
1255,MIL,decline,above 35
1256,LAC,decline,below 35
1257,BRK,decline,below 35
1258,LAC,decline,below 20


<h2>Perform Machine Learning with Opposition Team Feature Included</h2>

In [105]:
std_scaler = StandardScaler()

In [106]:
one_hot_encoded_sig_df = sig_df.select_dtypes(exclude=["object"])

In [107]:
target = full_game_df["+/-"]

In [108]:
one_hot_encoded_sig_df.columns

Index(['game_result', 'FG%', 'GmSc', 'TS%', 'GmSc-Per-PTS', 'PTS-Per-POSS',
       'game_result-Per-PTS', 'PER', 'game_location', 'Won',
       'High-3P%-Shooting', 'High-FG%-Shooting', 'Over-3-BLKs-and-STLs',
       'Played-Above-Avg-Min', 'Above-Avg-FT%', 'Above-Avg-PTS-Per-POSS',
       'More-BLKs-Than-STLs', 'More-AST-Than-TOV', '2P%-Above-50%',
       'More than 3 Fouls', 'ATL', 'BOS', 'BRK', 'CHA', 'CHI', 'CHO', 'CLE',
       'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA',
       'MIL', 'MIN', 'NJN', 'NOH', 'NOK', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI',
       'PHO', 'POR', 'SAC', 'SAS', 'SEA', 'TOR', 'UTA', 'WAS', 'decline',
       'prime', 'rising', 'above 35', 'below 20', 'below 35'],
      dtype='object')

In [109]:
std_scale_sig_data = std_scaler.fit_transform(one_hot_encoded_sig_df.values)

In [110]:
std_scale_sig_df = pd.DataFrame(std_scale_sig_data,columns=one_hot_encoded_sig_df.columns)

In [111]:
std_scale_sig_df.head()

,game_result,FG%,GmSc,TS%,GmSc-Per-PTS,PTS-Per-POSS,game_result-Per-PTS,PER,game_location,Won,...,SEA,TOR,UTA,WAS,decline,prime,rising,above 35,below 20,below 35
0,-1.420272,0.846395,0.318334,0.027535,0.942241,0.480218,-1.126608,0.098135,-1.009570,-1.406671,...,-0.079936,-0.213643,-0.161427,-0.205387,-0.386827,-0.766095,1.0,-0.439525,-0.456129,0.708370
1,-1.035821,-0.304266,-0.975332,-0.576938,-0.652382,-1.059284,-0.920912,-1.119288,-1.009570,-1.406671,...,-0.079936,-0.213643,-0.161427,-0.205387,-0.386827,-0.766095,1.0,-0.439525,-0.456129,0.708370
2,-1.804724,-2.275553,-2.230188,-2.455923,-1.067648,-2.435320,-3.967223,-2.240462,-1.009570,-1.406671,...,-0.079936,-0.213643,-0.161427,-0.205387,-0.386827,-0.766095,1.0,-0.439525,2.192362,-1.411692
3,-0.651370,-2.070397,-1.428115,-2.490924,4.330813,-2.094377,-1.144495,-1.473457,0.990521,-1.406671,...,-0.079936,-0.213643,-0.161427,-0.205387,-0.386827,-0.766095,1.0,-0.439525,2.192362,-1.411692
4,-0.420699,-0.545102,-1.712722,-0.371893,-2.361593,-1.142999,-0.318212,-1.924908,-1.009570,-1.406671,...,-0.079936,-0.213643,-0.161427,-0.205387,-0.386827,-0.766095,1.0,-0.439525,-0.456129,0.708370


In [112]:
std_scale_sig_df["+/-"] = target

In [113]:
print(f"There are {len(std_scale_sig_df.columns)} columns")
std_scale_sig_df.columns

There are 62 columns


Index(['game_result', 'FG%', 'GmSc', 'TS%', 'GmSc-Per-PTS', 'PTS-Per-POSS',
       'game_result-Per-PTS', 'PER', 'game_location', 'Won',
       'High-3P%-Shooting', 'High-FG%-Shooting', 'Over-3-BLKs-and-STLs',
       'Played-Above-Avg-Min', 'Above-Avg-FT%', 'Above-Avg-PTS-Per-POSS',
       'More-BLKs-Than-STLs', 'More-AST-Than-TOV', '2P%-Above-50%',
       'More than 3 Fouls', 'ATL', 'BOS', 'BRK', 'CHA', 'CHI', 'CHO', 'CLE',
       'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA',
       'MIL', 'MIN', 'NJN', 'NOH', 'NOK', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI',
       'PHO', 'POR', 'SAC', 'SAS', 'SEA', 'TOR', 'UTA', 'WAS', 'decline',
       'prime', 'rising', 'above 35', 'below 20', 'below 35', '+/-'],
      dtype='object')

In [114]:
ln = LinearRegression()

In [115]:
X_train, X_test, y_train, y_test = train_test_split(std_scale_sig_df.drop("+/-",axis=1), std_scale_sig_df["+/-"], test_size=0.33, random_state=42)

In [116]:
ln.fit(X_train,y_train)

LinearRegression()

In [117]:
y_pred = ln.predict(X_test)

In [118]:
rmse = mean_squared_error(y_test,y_pred,squared=False)
print(f"The root mean squared error is {round(rmse,2)} points")

The root mean squared error is 6.73 points


<h4>Perform PCA to see if there is any performance improvement</h4>

In [119]:
pca = PCA()

In [120]:
pca.fit_transform(one_hot_encoded_sig_df.values)

array([[ 1.22980724e+01, -1.40787345e+01,  2.42832188e-01, ...,
        -6.90215163e-15, -3.20754327e-15,  6.21976989e-16],
       [ 1.91805886e+01,  1.15836357e+00, -1.02894998e-01, ...,
        -5.37548508e-15,  2.84616663e-15,  3.73714869e-16],
       [ 3.66022808e+01,  5.70160028e+00,  1.21759167e+00, ...,
         8.65901084e-15, -1.50315547e-15, -2.67354734e-15],
       ...,
       [-2.94536778e-02, -9.42423770e+00,  3.17168525e+00, ...,
         2.48691146e-16,  4.29883964e-16, -1.64573798e-16],
       [ 1.52195511e+01,  1.34507127e+01,  4.33292335e-01, ...,
         1.10134257e-15, -7.19467988e-16,  7.15628363e-17],
       [ 2.22283309e+01, -4.13636504e+00,  2.64395164e+00, ...,
         5.91824373e-17, -7.99715897e-16,  6.07131875e-17]])

In [121]:
sum(pca.explained_variance_ratio_)

0.9999999999999999

In [122]:
cum_sum = np.cumsum(pca.explained_variance_ratio_)

In [123]:
cum_sum

array([0.67031867, 0.96991066, 0.9885964 , 0.98986853, 0.99085044,
       0.99170973, 0.99243473, 0.99313582, 0.9937931 , 0.99432719,
       0.99475935, 0.99518668, 0.99555267, 0.99586447, 0.99613883,
       0.99640877, 0.99664001, 0.99682893, 0.99701074, 0.99715406,
       0.99728677, 0.9974192 , 0.99754959, 0.99767852, 0.99780717,
       0.99793271, 0.99805723, 0.99817847, 0.99829773, 0.9984137 ,
       0.99851401, 0.99860365, 0.99868512, 0.99876566, 0.99884495,
       0.99892396, 0.99900215, 0.99907944, 0.99915482, 0.99922906,
       0.99930273, 0.99937534, 0.99944576, 0.99951497, 0.99958358,
       0.99965144, 0.99970973, 0.99975891, 0.99980061, 0.99983945,
       0.99987667, 0.99991203, 0.99993937, 0.99996275, 0.99998162,
       0.99999114, 0.99999847, 1.        , 1.        , 1.        ,
       1.        ])

In [124]:
np.argmax(cum_sum>0.95)

1

In [125]:
pca = PCA(n_components=2)

In [126]:
pca_transformed_sig_data = pca.fit_transform(one_hot_encoded_sig_df.values)

In [127]:
ln = LinearRegression()

In [128]:
X_train, X_test, y_train, y_test = train_test_split(pca_transformed_sig_data, std_scale_sig_df["+/-"], test_size=0.33, random_state=42)

In [129]:
ln.fit(X_train,y_train)

LinearRegression()

In [130]:
y_pred = ln.predict(X_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
print(f"The root mean squared error is {round(rmse,2)} points")

The root mean squared error is 6.68 points


<h2>Remove Opposing Team Columns</h2>

In [131]:
sig_df.columns

Index(['game_result', 'FG%', 'GmSc', 'TS%', 'GmSc-Per-PTS', 'PTS-Per-POSS',
       'game_result-Per-PTS', 'PER', 'game_location', 'Opp', 'Won',
       'High-3P%-Shooting', 'High-FG%-Shooting', 'Over-3-BLKs-and-STLs',
       'Played-Above-Avg-Min', 'Above-Avg-FT%', 'Career-Stage',
       'Above-Avg-PTS-Per-POSS', 'More-BLKs-Than-STLs', 'More-AST-Than-TOV',
       'PTS-Cat', '2P%-Above-50%', 'More than 3 Fouls', 'ATL', 'BOS', 'BRK',
       'CHA', 'CHI', 'CHO', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND',
       'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NJN', 'NOH', 'NOK', 'NOP',
       'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'SEA', 'TOR',
       'UTA', 'WAS', 'decline', 'prime', 'rising', 'above 35', 'below 20',
       'below 35'],
      dtype='object')

In [132]:
teams_lists = ['ATL', 'BOS', 'BRK',
       'CHA', 'CHI', 'CHO', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND',
       'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NJN', 'NOH', 'NOK', 'NOP',
       'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'SEA', 'TOR',
       'UTA', 'WAS']

In [133]:
non_opp_cols = list(set(sig_df.columns)-set(teams_lists))

In [134]:
sig_df_no_opp = sig_df[non_opp_cols].select_dtypes(exclude=["object"])

In [135]:
sig_df_no_opp.head()

,More than 3 Fouls,FG%,TS%,decline,GmSc-Per-PTS,PTS-Per-POSS,rising,above 35,More-AST-Than-TOV,Played-Above-Avg-Min,...,High-3P%-Shooting,below 20,GmSc,High-FG%-Shooting,below 35,game_result,PER,2P%-Above-50%,prime,Won
0,0.0,0.600,0.583431,0.0,0.988000,1.172608,1.0,0.0,1.0,1.0,...,0.0,0.0,24.7,1.0,1.0,-14.0,29.277048,1.0,0.0,0.0
1,0.0,0.471,0.516605,0.0,0.700000,0.837321,1.0,0.0,1.0,1.0,...,0.0,0.0,14.7,0.0,1.0,-9.0,16.893600,1.0,0.0,0.0
2,0.0,0.250,0.308880,0.0,0.625000,0.537634,1.0,0.0,1.0,1.0,...,0.0,1.0,5.0,0.0,0.0,-19.0,5.489179,0.0,0.0,0.0
3,0.0,0.273,0.305011,0.0,1.600000,0.611888,1.0,0.0,1.0,1.0,...,0.0,1.0,11.2,0.0,0.0,-4.0,13.291049,0.0,0.0,0.0
4,0.0,0.444,0.539273,0.0,0.391304,0.819088,1.0,0.0,0.0,1.0,...,1.0,0.0,9.0,0.0,1.0,-1.0,8.698953,0.0,0.0,0.0


In [136]:
std_scaler = StandardScaler()

In [137]:
sig_no_opp_data = std_scaler.fit_transform(sig_df_no_opp.values)

In [138]:
ln = LinearRegression()

In [139]:
X_train, X_test, y_train, y_test = train_test_split(sig_no_opp_data, std_scale_sig_df["+/-"], test_size=0.33, random_state=42)

In [140]:
ln.fit(X_train,y_train)

LinearRegression()

In [141]:
y_pred = ln.predict(X_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
print(f"The root mean squared error is {round(rmse,2)} points")

The root mean squared error is 6.64 points


<h4>Performance Analysis with PCA</h4>

In [142]:
pca = PCA()

In [143]:
sig_df_no_opp_w_pca = pca.fit_transform(sig_df_no_opp)

In [144]:
pca_cumsum = np.cumsum(pca.explained_variance_ratio_)

In [145]:
np.argmax(pca_cumsum>0.95)

1

In [146]:
pca = PCA(n_components=2)

In [147]:
sig_df_no_opp_w_pca = pca.fit_transform(sig_df_no_opp)

In [148]:
X_train, X_test, y_train, y_test = train_test_split(sig_df_no_opp_w_pca, std_scale_sig_df["+/-"], test_size=0.33, random_state=42)

In [149]:
ln = LinearRegression()

In [150]:
ln.fit(X_train,y_train)

LinearRegression()

In [151]:
y_pred = ln.predict(X_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
print(f"The root mean squared error is {round(rmse,2)} points")

The root mean squared error is 6.68 points
